<a href="https://colab.research.google.com/github/alrocks29/ddos-prediction-dnn/blob/master/DDos_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!/usr/bin/env python

In [0]:
#Import Liabraries
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns;
import pandas.util.testing as tm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import plot_model
from keras.models import Sequential, load_model
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils

<ipython-input-2-c836f8959645>:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [0]:
# #Cleaning Dataset and Exporting as cvs
# data_attack = pd.read_csv('dataset_attack.csv')
# data_normal = pd.read_csv('dataset_normal.csv')
# data_normal.columns=[ 'frame.len', 'frame.protocols', 'ip.hdr_len',
#        'ip.len', 'ip.flags.rb', 'ip.flags.df', 'p.flags.mf', 'ip.frag_offset',
#        'ip.ttl', 'ip.proto', 'ip.src', 'ip.dst', 'tcp.srcport', 'tcp.dstport',
#        'tcp.len', 'tcp.ack', 'tcp.flags.res', 'tcp.flags.ns', 'tcp.flags.cwr',
#        'tcp.flags.ecn', 'tcp.flags.urg', 'tcp.flags.ack', 'tcp.flags.push',
#        'tcp.flags.reset', 'tcp.flags.syn', 'tcp.flags.fin', 'tcp.window_size',
#        'tcp.time_delta','class']
# data_attack.columns=[ 'frame.len', 'frame.protocols', 'ip.hdr_len',
#        'ip.len', 'ip.flags.rb', 'ip.flags.df', 'p.flags.mf', 'ip.frag_offset',
#        'ip.ttl', 'ip.proto', 'ip.src', 'ip.dst', 'tcp.srcport', 'tcp.dstport',
#        'tcp.len', 'tcp.ack', 'tcp.flags.res', 'tcp.flags.ns', 'tcp.flags.cwr',
#        'tcp.flags.ecn', 'tcp.flags.urg', 'tcp.flags.ack', 'tcp.flags.push',
#        'tcp.flags.reset', 'tcp.flags.syn', 'tcp.flags.fin', 'tcp.window_size',
#        'tcp.time_delta','class']
# data_normal.to_csv (r'dataset_normal_cleaned.csv', index=False, header=True)
# data_attack.to_csv (r'dataset_attack_cleaned.csv', index=False, header=True)

In [0]:
data_normal = pd.read_csv('dataset_normal_cleaned.csv')
data_normal.head()

,frame.len,frame.protocols,ip.hdr_len,ip.len,ip.flags.rb,ip.flags.df,p.flags.mf,ip.frag_offset,ip.ttl,ip.proto,...,tcp.flags.ecn,tcp.flags.urg,tcp.flags.ack,tcp.flags.push,tcp.flags.reset,tcp.flags.syn,tcp.flags.fin,tcp.window_size,tcp.time_delta,class
0,206,eth:ethertype:ip:tcp:ssh,20,192,0,1,0,0,128,6,...,0,0,1,1,0,0,0,16697,0.000000,normal
1,60,eth:ethertype:ip:tcp,20,40,0,1,0,0,64,6,...,0,0,1,0,0,0,0,7504,0.000537,normal
2,60,eth:ethertype:ip:tcp,20,40,0,1,0,0,63,6,...,0,0,1,0,0,0,0,7504,0.000155,normal
3,774,eth:ethertype:ip:tcp:ssh,20,760,0,1,0,0,64,6,...,0,0,1,1,0,0,0,7504,0.004483,normal
4,774,eth:ethertype:ip:tcp,20,760,0,1,0,0,63,6,...,0,0,1,1,0,0,0,7504,0.001321,normal


In [0]:
data_attack = pd.read_csv('dataset_attack_cleaned.csv')
data_attack.head()

,frame.len,frame.protocols,ip.hdr_len,ip.len,ip.flags.rb,ip.flags.df,p.flags.mf,ip.frag_offset,ip.ttl,ip.proto,...,tcp.flags.ecn,tcp.flags.urg,tcp.flags.ack,tcp.flags.push,tcp.flags.reset,tcp.flags.syn,tcp.flags.fin,tcp.window_size,tcp.time_delta,class
0,212,eth:ethertype:ip:udp:data,20,198,0,0,0,0,112,17,...,0,0,0,0,0,0,0,0,0.0,attack
1,212,eth:ethertype:ip:udp:data,20,198,0,0,0,0,112,17,...,0,0,0,0,0,0,0,0,0.0,attack
2,212,eth:ethertype:ip:udp:data,20,198,0,0,0,0,112,17,...,0,0,0,0,0,0,0,0,0.0,attack
3,212,eth:ethertype:ip:udp:data,20,198,0,0,0,0,112,17,...,0,0,0,0,0,0,0,0,0.0,attack
4,62,eth:ethertype:ip:udp:data,20,48,0,0,0,0,128,17,...,0,0,0,0,0,0,0,0,0.0,attack


In [0]:
data_normal=data_normal.drop(['ip.src', 'ip.dst','frame.protocols'],axis=1)
data_attack=data_attack.drop(['ip.src', 'ip.dst','frame.protocols'],axis=1)

KeyError: "['ip.src' 'ip.dst' 'frame.protocols'] not found in axis"

In [0]:
features = data_attack.head(0)
features=data_attack.columns.tolist()
print(features)

['frame.len', 'ip.hdr_len', 'ip.len', 'ip.flags.rb', 'ip.flags.df', 'p.flags.mf', 'ip.frag_offset', 'ip.ttl', 'ip.proto', 'tcp.srcport', 'tcp.dstport', 'tcp.len', 'tcp.ack', 'tcp.flags.res', 'tcp.flags.ns', 'tcp.flags.cwr', 'tcp.flags.ecn', 'tcp.flags.urg', 'tcp.flags.ack', 'tcp.flags.push', 'tcp.flags.reset', 'tcp.flags.syn', 'tcp.flags.fin', 'tcp.window_size', 'tcp.time_delta', 'class']


In [0]:
X_normal= data_normal[features].values
X_attack= data_attack[features].values
Y_normal= data_normal['class']
Y_attack= data_attack['class']
X=np.concatenate((X_normal,X_attack))
Y=np.concatenate((Y_normal,Y_attack))

FileNotFoundError: [Errno 2] No such file or directory: 'data_normal_cleaned.csv'